In [ ]:
import os
import re
import pandas as pd
import spacy
from spacy.lang.en import English
from datetime import datetime
import json

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Remove language_tool_python dependency and implement a simpler spell checker
def simple_spell_check(text):
    """
    A simplified spell checker that looks for common patterns of errors
    Returns a rough estimate of text quality
    """
    # Look for common error patterns
    potential_errors = [
        r'\b\w*([a-z]{2,})\1\w*\b',  # Repeated letter patterns
        r'\b\w*[^a-zA-Z\s\'\-]\w*\b',  # Invalid characters
        r'\s{2,}',  # Multiple spaces
        r'[A-Z]{2,}[a-z]+'  # Improper capitalization
    ]
    
    error_count = 0
    for pattern in potential_errors:
        errors = re.findall(pattern, text)
        error_count += len(errors)
    
    total_words = len(text.split())
    if total_words == 0:
        return 0
    
    # Return a ratio between 0 and 1, where 1 is best
    return max(0, min(1, 1 - (error_count / total_words)))

def load_and_clean_data(directory):
    """Load data from text files in the specified directory and perform initial cleaning."""
    data = []
    for filename in os.listdir(directory):
        if filename.startswith("Resume_of_ID_") and filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
            id_number = int(filename.split('_')[3].split('.')[0])
            clean_text = ' '.join(text.split())  # Remove extra whitespace
            data.append({'ID': id_number, 'Text': clean_text})
    return pd.DataFrame(data)

def preprocess_text(text):
    """Preprocess text using spaCy for tokenization and lemmatization."""
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return tokens

def extract_years_of_experience(text):
    """Extract years of experience from the resume text."""
    years = re.findall(r'\b(19[7-9]\d|20[0-2]\d)\b', text)
    if len(years) >= 2:
        earliest_year = min(int(year) for year in years)
        latest_year = max(int(year) for year in years)
        current_year = datetime.now().year
        if latest_year > current_year:
            latest_year = current_year
        return latest_year - earliest_year
    return 0

def detect_education_level(text):
    """Detect the highest education level mentioned in the resume."""
    education_patterns = {
        'PhD': r'\bPh\.?D\.?\b|\bDoctor(ate)?\b',
        'Master': r'\bM\.?S\.?\b|\bM\.?A\.?\b|\bMaster\b',
        'Bachelor': r'\bB\.?S\.?\b|\bB\.?A\.?\b|\bBachelor\b',
        'Associate': r'\bA\.?S\.?\b|\bA\.?A\.?\b|\bAssociate\b'
    }

    for level, pattern in education_patterns.items():
        if re.search(pattern, text, re.IGNORECASE):
            return level
    return 'Other'

def identify_resume_sections(text):
    """Identify and score the presence of important resume sections."""
    important_sections = ['education', 'experience', 'skills', 'projects', 'achievements']
    optional_sections = ['summary', 'objective', 'interests', 'activities']
    unnecessary_sections = ['references']

    section_score = 0
    for section in important_sections:
        if re.search(r'\b' + section + r'\b', text, re.IGNORECASE):
            section_score += 1

    for section in optional_sections:
        if re.search(r'\b' + section + r'\b', text, re.IGNORECASE):
            section_score += 0.5

    for section in unnecessary_sections:
        if re.search(r'\b' + section + r'\b', text, re.IGNORECASE):
            section_score -= 0.5

    return min(section_score / len(important_sections), 1)

def quantify_brevity(text):
    """Quantify the brevity of the resume."""
    word_count = len(text.split())
    if word_count < 200:
        return 0.5  # Too short
    elif word_count > 1000:
        return 0.5  # Too long
    else:
        return 1 - (abs(600 - word_count) / 400)  # Optimal around 600 words

def process_resume(row):
    """Process a single resume and return a dictionary of features."""
    text = row['Text']
    tokens = preprocess_text(text)

    years_of_experience = extract_years_of_experience(text)
    education_level = detect_education_level(text)
    spell_check_ratio = simple_spell_check(text)  # Using our simplified spell checker
    section_score = identify_resume_sections(text)
    brevity_score = quantify_brevity(text)

    return {
        'ID': row['ID'],
        'Preprocessed_Tokens': tokens,
        'Years_of_Experience': years_of_experience,
        'Education_Level': education_level,
        'Spell_Check_Ratio': spell_check_ratio,
        'Section_Score': section_score,
        'Brevity_Score': brevity_score
    }

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from textblob import TextBlob

def calculate_word_sentence_counts(text):
    """Calculate word count and sentence count."""
    sentences = re.split(r'[.!?]+', text)
    word_count = len(text.split())
    sentence_count = len([s for s in sentences if s.strip()])
    return word_count, sentence_count

def extract_skills(text, skill_list):
    """Extract skills from text based on a predefined skill list."""
    found_skills = [skill for skill in skill_list if re.search(r'\b' + re.escape(skill) + r'\b', text, re.IGNORECASE)]
    return found_skills

def calculate_skill_match_score(resume_skills, job_skills):
    """Calculate the skill match score."""
    if not job_skills:
        return 0
    matched_skills = set(resume_skills) & set(job_skills)
    return len(matched_skills) / len(job_skills)

def analyze_sentiment(text):
    """Analyze the sentiment of achievement statements in the resume."""
    blob = TextBlob(text)
    return blob.sentiment.polarity

def quantify_achievement_impact(text):
    """Quantify the impact of achievements."""
    impact_score = 0
    achievements = re.findall(r'\b(increased|decreased|improved|reduced|saved|generated).*?(\d+(?:\.\d+)?%?)', text, re.IGNORECASE)
    for _, value in achievements:
        if '%' in value:
            impact_score += float(value.strip('%')) / 100
        else:
            impact_score += float(value) / 1000  # Assume larger numbers for non-percentage values
    return min(impact_score, 1)

def calculate_technical_score(row):
    """Calculate the technical CV score."""
    skill_count = len(row['Extracted_Skills'])
    experience_score = min(row['Years_of_Experience'] / 10, 1)  # Cap at 10 years
    education_score = {'PhD': 1, 'Master': 0.8, 'Bachelor': 0.6, 'Associate': 0.4, 'Other': 0.2}.get(row['Education_Level'], 0.2)

    return (skill_count / 10 * 0.4 + experience_score * 0.3 + education_score * 0.3)

def calculate_managerial_score(row):
    """Calculate the managerial CV score."""
    soft_skills_score = analyze_sentiment(row['Text'])
    achievement_impact = quantify_achievement_impact(row['Text'])
    leadership_score = min(row['Years_of_Experience'] / 15, 1)  # Assume leadership potential increases with experience

    return (soft_skills_score * 0.3 + achievement_impact * 0.4 + leadership_score * 0.3)

def calculate_overall_score(row):
    """Calculate the overall CV score."""
    technical_score = row['Technical_Score']
    managerial_score = row['Managerial_Score']
    resume_quality_score = (row['Spell_Check_Ratio'] + row['Section_Score'] + row['Brevity_Score']) / 3
    return (technical_score * 0.4 + managerial_score * 0.3 + resume_quality_score * 0.3)

def process_resume_section2(row, job_skills):
    """Process a single resume for Section 2 analysis."""
    word_count, sentence_count = calculate_word_sentence_counts(row['Text'])

    # Combine general and technical skills
    all_skills = job_skills['general_skills'] + job_skills['technical_skills']

    # Extract skills based on the generated job description
    job_specific_skills = extract_skills(row['Generated_Job_Description'], all_skills)
    resume_skills = extract_skills(row['Text'], all_skills)

    technical_score = calculate_technical_score(row, job_specific_skills)
    managerial_score = calculate_managerial_score(row)
    resume_quality_score = (row['Spell_Check_Ratio'] + row['Section_Score'] + row['Brevity_Score']) / 3
    overall_score = calculate_overall_score(technical_score, managerial_score, resume_quality_score)

    return {
        'ID': row['ID'],
        'Word_Count': word_count,
        'Sentence_Count': sentence_count,
        'Resume_Skills': resume_skills,
        'Job_Specific_Skills': job_specific_skills,
        'Technical_Score': technical_score,
        'Managerial_Score': managerial_score,
        'Resume_Quality_Score': resume_quality_score,
        'Overall_Score': overall_score
    }

def normalize_scores(df):
    """Normalize scores to ensure fair comparison across all resumes."""
    score_columns = ['Technical_Score', 'Managerial_Score', 'Resume_Quality_Score', 'Overall_Score']
    for column in score_columns:
        df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    return df

def process_resumes_section2(df, job_skills):
    """Process all resumes for Section 2 analysis."""
    results = []
    for _, row in df.iterrows():
        results.append(process_resume_section2(row, job_skills))

    results_df = pd.DataFrame(results)
    normalized_df = normalize_scores(results_df)
    return normalized_df

# This function will be called from the main function in Section 3
def run_section2(input_file, job_skills):
    """Run Section 2 processing on the input file."""
    df = pd.read_csv(input_file)
    processed_df = process_resumes_section2(df, job_skills)
    processed_df.to_csv('processed_resumes_section2.csv', index=False)
    print("Section 2 processing completed. Results saved to 'processed_resumes_section2.csv'")
    return processed_df

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from typing import List, Dict

def load_job_skills(file_path: str) -> List[str]:
    """Load job skills from a JSON file."""
    with open(file_path, 'r') as file:
        return json.load(file)

def job_description_matching(resume_text: str, job_description: str) -> float:
    """Calculate similarity between resume and job description."""
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([resume_text, job_description])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

def adjust_scores_with_job_match(df: pd.DataFrame, job_description: str) -> pd.DataFrame:
    """Adjust scores based on job description matching."""
    df['Job_Match_Score'] = df['Text'].apply(lambda x: job_description_matching(x, job_description))
    df['Adjusted_Overall_Score'] = (df['Overall_Score'] * 0.7 + df['Job_Match_Score'] * 0.3)
    return df

def rank_resumes(df: pd.DataFrame) -> pd.DataFrame:
    """Rank resumes based on adjusted overall score."""
    return df.sort_values('Adjusted_Overall_Score', ascending=False).reset_index(drop=True)

def generate_report(df: pd.DataFrame, top_n: int = 10) -> Dict:
    """Generate a report with top candidates and summary statistics."""
    top_candidates = df.head(top_n)[['ID', 'Adjusted_Overall_Score', 'Technical_Score', 'Managerial_Score', 'Job_Match_Score']]
    summary_stats = df[['Adjusted_Overall_Score', 'Technical_Score', 'Managerial_Score', 'Job_Match_Score']].describe()

    return {
        'top_candidates': top_candidates.to_dict(orient='records'),
        'summary_stats': summary_stats.to_dict()
    }

def match_resume_to_job_description(resume_text, job_description):
    """Match a resume to a specific job description and return adjusted scores."""
    # Reuse the existing job_description_matching function
    match_score = job_description_matching(resume_text, job_description)

    # Recalculate scores based on the new job description
    job_specific_skills = extract_skills(job_description, general_job_skills)
    resume_skills = extract_skills(resume_text, job_specific_skills)

    technical_score = calculate_technical_score({'Text': resume_text, 'Years_of_Experience': extract_years_of_experience(resume_text), 'Education_Level': detect_education_level(resume_text)}, job_specific_skills)
    managerial_score = calculate_managerial_score({'Text': resume_text})
    resume_quality_score = (simple_spell_check(resume_text) + identify_resume_sections(resume_text) + quantify_brevity(resume_text)) / 3
    overall_score = calculate_overall_score(technical_score, managerial_score, resume_quality_score)

    adjusted_overall_score = overall_score * 0.7 + match_score * 0.3

    return {
        'Technical_Score': technical_score,
        'Managerial_Score': managerial_score,
        'Resume_Quality_Score': resume_quality_score,
        'Overall_Score': overall_score,
        'Job_Match_Score': match_score,
        'Adjusted_Overall_Score': adjusted_overall_score
    }

def create_default_job_skills_file(file_path):
    """Create a default job_skills.json file if it doesn't exist."""
    default_skills = {
        "general_skills": [
            "communication", "teamwork", "leadership", "problem-solving",
            "time management", "analytical skills", "creativity", "adaptability"
        ],
        "technical_skills": [
            "programming", "data analysis", "project management",
            "software development", "database management", "web development"
        ]
    }

    with open(file_path, 'w') as file:
        json.dump(default_skills, file, indent=2)

import json
import os

def load_job_skills(file_path: str) -> List[str]:
    """Load general job skills from a JSON file or use a default list."""
    default_skills = [
        "communication", "teamwork", "leadership", "problem-solving",
        "time management", "analytical skills", "creativity", "adaptability",
        "organization", "attention to detail", "customer service",
        "critical thinking", "decision making", "interpersonal skills",
        "multitasking", "flexibility", "initiative", "reliability",
        "professionalism", "continuous learning"
    ]

    try:
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                skills_data = json.load(file)
            if isinstance(skills_data, list):
                return skills_data
            elif isinstance(skills_data, dict) and 'skills' in skills_data:
                return skills_data['skills']
            else:
                print(f"Unexpected format in '{file_path}'. Using default skills list.")
                return default_skills
        else:
            print(f"'{file_path}' not found. Using default skills list.")
            return default_skills
    except json.JSONDecodeError:
        print(f"Error reading '{file_path}'. Using default skills list.")
        return default_skills

def extract_skills(text: str) -> List[str]:
    """Extract skills from text using NLP techniques."""
    # This is a placeholder function. In a real implementation, you would use
    # more sophisticated NLP techniques to extract skills from the text.
    # For now, we'll use a simple keyword matching approach.
    extracted_skills = []
    for skill in general_skills:
        if skill.lower() in text.lower():
            extracted_skills.append(skill)
    return extracted_skills

def process_resume(row):
    """Process a single resume and return a dictionary of features."""
    text = row['Text']

    years_of_experience = extract_years_of_experience(text)
    education_level = detect_education_level(text)
    spell_check_ratio = simple_spell_check(text)
    section_score = identify_resume_sections(text)
    brevity_score = quantify_brevity(text)
    extracted_skills = extract_skills(text)

    return {
        'ID': row['ID'],
        'Years_of_Experience': years_of_experience,
        'Education_Level': education_level,
        'Spell_Check_Ratio': spell_check_ratio,
        'Section_Score': section_score,
        'Brevity_Score': brevity_score,
        'Extracted_Skills': extracted_skills
    }

def main(resume_directory: str):
    print("Starting resume analysis process...")

    # Load general skills
    global general_skills
    general_skills = load_job_skills('job_skills.json')

    # Load and preprocess data
    print("Loading and preprocessing resumes...")
    df = load_and_clean_data(resume_directory)
    df['processed'] = df.apply(process_resume, axis=1)  #here is the problem of language_tool
    df = pd.concat([df, pd.DataFrame(df['processed'].tolist())], axis=1)
    df.drop('processed', axis=1, inplace=True)

    # Calculate scores
    df['Skill_Count'] = df['Extracted_Skills'].apply(len)
    df['Technical_Score'] = df.apply(calculate_technical_score, axis=1)
    df['Managerial_Score'] = df.apply(calculate_managerial_score, axis=1)
    df['Overall_Score'] = df.apply(calculate_overall_score, axis=1)

    # Rank resumes
    print("Ranking resumes...")
    ranked_df = df.sort_values('Overall_Score', ascending=False).reset_index(drop=True)

    # Save final results
    final_columns = [
        'ID', 'Years_of_Experience', 'Education_Level', 'Spell_Check_Ratio',
        'Section_Score', 'Brevity_Score', 'Skill_Count',
        'Technical_Score', 'Managerial_Score', 'Overall_Score', 'Extracted_Skills']
    ranked_df[final_columns].to_csv('final_ranked_resumes.csv', index=False)

    print("Resume analysis complete. Results saved to 'final_ranked_resumes.csv'")

    return ranked_df

if __name__ == "__main__":
    resume_directory = 'extracted_text_files/'
    main(resume_directory)